In [5]:
import pommerman
from pommerman import agents, make

print(pommerman.REGISTRY)

['PommeFFACompetition-v0', 'PommeFFACompetitionFast-v0', 'PommeFFAFast-v0', 'PommeFFA-v1', 'OneVsOne-v0', 'PommeRadioCompetition-v2', 'PommeRadio-v2', 'PommeTeamCompetition-v0', 'PommeTeamCompetitionFast-v0', 'PommeTeamCompetition-v1', 'PommeTeam-v0', 'PommeTeamFast-v0']


In [12]:
import inspect

inspect.getfullargspec(make)

FullArgSpec(args=['config_id', 'agent_list', 'game_state_file', 'render_mode'], varargs=None, varkw=None, defaults=(None, 'human'), kwonlyargs=[], kwonlydefaults=None, annotations={})

In [14]:
print(inspect.getsource(make))

def make(config_id, agent_list, game_state_file=None, render_mode='human'):
    '''Makes the pommerman env and registers it with gym'''
    assert config_id in REGISTRY, "Unknown configuration '{}'. " \
        "Possible values: {}".format(config_id, REGISTRY)
    env = gym.make(config_id)

    for id_, agent in enumerate(agent_list):
        assert isinstance(agent, agents.BaseAgent)
        # NOTE: This is IMPORTANT so that the agent character is initialized
        agent.init_agent(id_, env.spec._kwargs['game_type'])

    env.set_agents(agent_list)
    env.set_init_game_state(game_state_file)
    env.set_render_mode(render_mode)
    return env



## How does the environment work?

\<class pommerman.envs.vo.Pomme\> --> self.model = \<class pommerman.forward_model.ForwardModel\>
\<class pommerman.envs.vo.Pomme\> --> 


In [31]:
from pommerman.envs.v0 import Pomme
print(inspect.getsource(Pomme.reset))
print(inspect.getsource(Pomme.get_observations))
print(inspect.getsource(Pomme.__init__))

    def reset(self):
        assert (self._agents is not None)

        if self._init_game_state is not None:
            self.set_json_info()
        else:
            self._step_count = 0
            self.make_board()
            self.make_items()
            self._bombs = []
            self._flames = []
            self._powerups = []
            for agent_id, agent in enumerate(self._agents):
                pos = np.where(self._board == utility.agent_value(agent_id))
                row = pos[0][0]
                col = pos[1][0]
                agent.set_start_position((row, col))
                agent.reset()

        return self.get_observations()

    def get_observations(self):
        self.observations = self.model.get_observations(
            self._board, self._agents, self._bombs, self._flames,
            self._is_partially_observable, self._agent_view_size,
            self._game_type, self._env)
        for obs in self.observations:
            obs['step_count'] = sel

In [19]:
env = make('OneVsOne-v0',[agents.SimpleAgent(), agents.SimpleAgent()])

## observation
- The observation is given by the get_observations method from ForwardModel. (get_observations method in the v0.py calls the get_observation method in forward_model.py

- This is the global observation

In [44]:
state = env.reset()
print(state)
env.render()
done = False


[{'alive': [10, 11], 'board': array([[ 0,  2,  2,  1,  1,  0,  0,  0],
       [ 2, 10,  0,  0,  2,  2,  0,  2],
       [ 2,  0,  0,  2,  0,  0,  0,  0],
       [ 1,  0,  2,  0,  2,  1,  1,  1],
       [ 1,  2,  0,  2,  0,  2,  0,  1],
       [ 0,  2,  0,  1,  2,  0,  0,  1],
       [ 0,  0,  0,  1,  0,  0, 11,  1],
       [ 0,  2,  0,  1,  1,  1,  1,  0]], dtype=uint8), 'bomb_blast_strength': array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]]), 'bomb_life': array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0.,

In [47]:
actions = env.act(state)
print(actions)
state, reward, done, info = env.step(actions)
import pprint
pprint.pprint([state, reward, done, info])

[4, 1]
[[{'alive': [10, 11],
   'ammo': 0,
   'blast_strength': 2,
   'board': array([[ 0,  2,  2,  1,  1,  0,  0,  0],
       [ 2,  3, 10,  0,  2,  2,  0,  2],
       [ 2,  0,  0,  2,  0,  0,  0,  0],
       [ 1,  0,  2,  0,  2,  1,  1,  1],
       [ 1,  2,  0,  2,  0,  2,  0,  1],
       [ 0,  2,  0,  1,  2, 11,  0,  1],
       [ 0,  0,  0,  1,  0,  0,  0,  1],
       [ 0,  2,  0,  1,  1,  1,  1,  0]], dtype=uint8),
   'bomb_blast_strength': array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]]),
   'bomb_life': array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 8., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
 

In [48]:
env.render()